In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
import warnings
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch
import numpy as np
from IPython.display import Audio

# GPU 사용 여부 확인 및 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125", temperature=0)

#음악 설명 생성
def generate_response(mood, genre, elements, tempo):
    description = f"현재 기분: {mood}, 선호하는 음악 장르: {genre}, 음악 요소: {elements}, 분위기/속도: {tempo}"
    full_prompt = f"""
    
    당신은 ADHD 환자를 위한 음악 설명을 작성하는 어시스턴트입니다. 
    사용자의 입력을 바탕으로 음악 설명을 작성해야합니다.
    사용자의 기분, 상황, 선호 장르, 분위기/속도에 맞춰 집중력을 향상시키고 스트레스를 감소시키는 음악의 설명을 작성해주세요.
    음악 설명은 다음의 예시의 형식대로 작성해야합니다. 

    ##예시
    이 음악은 차분하고 느린 템포의 클래식 음악으로, 바쁜 일상 속에서 집중력을 높이고 스트레스를 줄이는 데 도움을 줄 수 있습니다. 
    피아노와 바이올린의 부드러운 선율이 조화를 이루며, 마음을 편안하게 해줍니다. 
    특히 바흐의 '골드베르크 변주곡'처럼 복잡하지 않으면서도 집중력을 향상시키는 음악이 이상적입니다. 
    이 음악을 들으면 중요한 프로젝트를 완성하는 동안 마음의 안정을 찾고, 차분하게 일에 몰두할 수 있을 것입니다.

    {description}
    """
    result = llm.invoke(full_prompt)
    
    if hasattr(result, 'content'):
        return result.content
    elif hasattr(result, 'text'):
        return result.text
    else:
        return "No text content found in the response."

# 사용자 입력
def get_user_input():
    print("ADHD 환자를 위한 집중력 향상과 스트레스 감소를 위한 음악을 생성하는 어시스턴트입니다.")
    print("다음 정보를 제공해주세요:")
    mood = input("1. 현재 기분 또는 상황을 입력하세요: ")
    genre = input("2. 선호하는 음악 장르를 입력하세요: ")
    elements = input("3. 음악에 반영되었으면 하는 요소를 입력하세요 (예: 자연 소리, 특정 악기 등): ")
    tempo = input("4. 원하는 음악의 분위기나 속도를 입력하세요: ")
    return mood, genre, elements, tempo

mood, genre, elements, tempo = get_user_input()
response = generate_response(mood, genre, elements, tempo)
print("생성된 음악 설명:")
print(response)

# 음악 생성 모델을 위한 음악 설명 요약 및 번역
def music_features(response):
    summary_prompt = f"""
    요약: 이 음악 설명에서 음악의 특징적인 부분만을 30자 이내로 간략하게 서술해주세요. 
    음악을 통해 얻을 수 있는 효과는 제거하고, 음악적 특성만을 서술해주세요. 
    답변은 영어로 출력합니다. 
    아래의 형식처럼 음악의 특성만을 고려하여 작성하고, 한줄로 출력해주세요.

    출력 형식:
    a catchy beat for a podcast intro
    
    예시: 
    1. a catchy beat for a podcast intro
    2. a funky house with 80s hip hop vibes
    3. a chill song with influences from lofi, chillstep and downtempo
    \n\n'{response}'"""
    # 요약 요청을 모델에 전달
    summary_result = llm.invoke(summary_prompt)
    return summary_result

# 음악 특성 요약
summary = music_features(response).content
print("요약된 음악 특징:")
print(summary)


# 음악 생성 모델
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device)

def generate_music(summary, duration_seconds, base_tokens=256, base_duration=5):
    # Musicgen 모델을 사용하여 음악 생성
    # 주어진 초 단위 시간을 바탕으로 max_new_tokens 값을 계산
    tokens_per_second = base_tokens / base_duration
    max_new_tokens = int(tokens_per_second * duration_seconds)

    inputs = processor(text=[summary], padding=True, return_tensors="pt").to(device)
    audio_values = model.generate(**inputs, max_new_tokens=max_new_tokens)
    sampling_rate = model.config.audio_encoder.sampling_rate
    
    # 오디오 데이터를 스트리밍으로 반환
    audio_data = audio_values[0, 0].cpu().numpy()
    
    return audio_data, sampling_rate

# 사용자로부터 원하는 음악 길이를 초 단위로 입력받음
duration_seconds = int(input("생성할 음악의 길이를 초 단위로 입력하세요: "))

# 음악 생성
audio_data, sampling_rate = generate_music(summary, duration_seconds, base_tokens=256, base_duration=5)


# 오디오 데이터 재생 (IPython Audio 객체를 사용하여 재생)
Audio(data=audio_data, rate=sampling_rate)